# Convert OpenSfM reconstruction to format used by our tool

In [1]:
import sys
import os
import re
import csv
import json
import copy
import math
import random
import pickle
from itertools import combinations
from collections import defaultdict
import numpy as np
import cv2
import networkx as nx
import matplotlib.pyplot as plt
import g2o
from scipy.spatial.distance import pdist, squareform
import tqdm

sys.path.append("../..")
from extractor.mapping.geotransforms import enu2geodetic, geodetic2enu
from extractor.mapping.geometry import from_twist
from extractor.common import get_immediate_subdirectories

In [2]:
work_dir = "/storage-2/pvextractor-georeferencing/20210510_Schmalenbach/workdir/02_north"
mapping_root = os.path.join(work_dir, "mapping")

# find all clusters
reconstruction_files = []
tracks_file = os.path.join(work_dir, "tracking", "tracks.csv")
patches_meta_file = os.path.join(work_dir, "patches", "meta.pkl")

for cluster_dir in sorted(get_immediate_subdirectories(mapping_root)):
    reconstruction_file = os.path.join(mapping_root, cluster_dir, "reconstruction.json")
    
    if os.path.isfile(reconstruction_file):
        reconstruction_files.append(reconstruction_file)
        
assert len(reconstruction_files) > 0, "No reconstructions found. Run OpenSfM first."

In [3]:
reconstructions = []
for file in reconstruction_files:
    reconstructions.extend(json.load(open(file, "r")))

print("Number of reconstructions: {}".format(len(reconstructions)))

Number of reconstructions: 3


### Equalize reference points of reconstructions

When combining multiple reconstructions each one uses its own ECEF origin. To equalize them, we select the origin of the first reconstruction and convert all other reconstructions to this origin.

In [4]:
overall_origin = tuple(reconstructions[0]["reference_lla"].values())
pickle.dump(overall_origin, open(os.path.join(mapping_root, "reference_lla.pkl"), "wb"))

In [5]:
# convert reconstruction from ECEF to WGS-84 using its own origin
# pick one of the origins as overall origin, e.g. pick the one of the first reconstruction
# convert reconstruction back to ECEF using the overall origin
# Note: This may take a while.
for reconstruction in reconstructions:
    origin = tuple(reconstruction["reference_lla"].values())
    if origin == overall_origin:
        continue
    
    # transform map points
    for i, point_id in enumerate(reconstruction["points"].keys()):
        geo = enu2geodetic(*reconstruction["points"][point_id]["coordinates"], *origin)
        transformed_ecef = geodetic2enu(*geo, *overall_origin)
        reconstruction["points"][point_id]["coordinates"] = transformed_ecef

In [6]:
for reconstruction in reconstructions:
    origin = tuple(reconstruction["reference_lla"].values())
    if origin == overall_origin:
        continue
    
    # transform camera poses
    for i, shot_id in enumerate(reconstruction["shots"].keys()):
        shot = reconstruction["shots"][shot_id]
        R, _ = cv2.Rodrigues(np.array(shot["rotation"]))
        t = np.array(shot["translation"])
        t = -R.T.dot(t)        
        geo = enu2geodetic(*t, *origin)
        transformed_ecef = geodetic2enu(*geo, *overall_origin)        
        transformed_ecef = -1.0 * (R @ np.array(transformed_ecef).reshape(3, 1)).reshape(3,)
        reconstruction["shots"][shot_id]["translation"] = transformed_ecef

### Convert map points

In [7]:
map_points = []
for reconstruction in reconstructions:
    for i, point in enumerate(reconstruction['points'].values()):
        map_points.append(point['coordinates'])
map_points = np.vstack(map_points)
pickle.dump(map_points, open(os.path.join(mapping_root, "map_points.pkl"), "wb"))

### Convert camera poses

In [8]:
pose_graphs = []
i = 0
for reconstruction in reconstructions:
    pose_graph = nx.Graph()
    for frame_name in sorted(reconstruction['shots']):
        shot = reconstruction['shots'][frame_name]
        R, _ = cv2.Rodrigues(np.array(shot["rotation"]))
        t = np.array(shot["translation"])
        t = -R.T.dot(t)
        R, _ = cv2.Rodrigues(R.T)
        pose = np.hstack((R.reshape(3,), t))
        frame_name = str.split(frame_name, ".")[0]
        pose_graph.add_node(frame_name, pose=pose)
        i += 1
    pose_graphs.append(pose_graph)
    
pose_graph = nx.compose_all(pose_graphs)
pickle.dump(pose_graph, open(os.path.join(mapping_root, "pose_graph.pkl"), "wb"))

### Convert camera matrix and distortion coefficients

In [9]:
def convert_camera_matrix(reconstruction, camera_model="brown"):
    """Convert camera matrix from OpenSfM format to OpenCV format 
    (pixel units). Needed because OpenSfM modifies camera parameters 
    during the reconstruction."""
    
    if camera_model == "brown":
        camera = list(reconstruction['cameras'].keys())[0]
        image_width = reconstruction['cameras'][camera]['width']
        image_height = reconstruction['cameras'][camera]['height']
        scale = np.maximum(image_width, image_height)
        fx = reconstruction['cameras'][camera]['focal_x'] * scale
        fy = reconstruction['cameras'][camera]['focal_y'] * scale
        cx = reconstruction['cameras'][camera]['c_x'] * scale + 0.5*image_width
        cy = reconstruction['cameras'][camera]['c_y'] * scale + 0.5*image_height

        camera_matrix = np.array([[fx, 0.0, cx],
                                  [0.0, fy, cy],
                                  [0.0, 0.0, 1.0]])

        k1 = reconstruction['cameras'][camera]['k1']
        k2 = reconstruction['cameras'][camera]['k2']
        p1 = reconstruction['cameras'][camera]['p1']
        p2 = reconstruction['cameras'][camera]['p2']
        k3 = reconstruction['cameras'][camera]['k3']
        dist_coeffs = np.array([[k1, k2, p1, p2, k3]])
        
    elif camera_model == "perspective":
        dist_coeffs = None
        camera = list(reconstruction['cameras'].keys())[0]
        image_width = reconstruction['cameras'][camera]['width']
        image_height = reconstruction['cameras'][camera]['height']
        scale = np.maximum(image_width, image_height)
        focal = reconstruction['cameras'][camera]['focal'] * scale
        cx = image_width / 2
        cy = image_height / 2
        camera_matrix = np.array([[focal, 0.0, cx],
                                  [0.0, focal, cy],
                                  [0.0, 0.0, 1.0]])
    
    return camera_matrix, dist_coeffs

In [10]:
# insert camera matrix and distortion coefficients into pose graph
for reconstruction in reconstructions:
    camera_matrix, dist_coeffs = convert_camera_matrix(reconstruction)
    for frame_name in sorted(reconstruction['shots']):
        frame_name = str.split(frame_name, ".")[0]
        pose_graph.nodes[frame_name]["camera_matrix"] = camera_matrix
        pose_graph.nodes[frame_name]["dist_coeffs"] = dist_coeffs

### Reduce size of reconstruction for faster development

### Triangulate PV module corners and centers

In [11]:
def save_modules(module_corners):
    module_centers_ = {track_id: pts_3d[0, :].reshape(1, 3) for track_id, pts_3d in module_corners.items()}
    module_corners_ = {track_id: pts_3d[1:, :].reshape(-1, 3) for track_id, pts_3d in module_corners.items()}

    modules = {
        "corners": module_corners_,
        "centers": module_centers_
    }
    pickle.dump(modules, open(os.path.join(mapping_root, "modules.pkl"), "wb"))
    

def save_modules_geocoords(module_corners, gps_origin):
    """Save WGS-84 coordinates of PV modules in GeoJSON file."""
    module_centers_ = {track_id: pts_3d[0, :].reshape(1, 3) for track_id, pts_3d in module_corners.items()}
    module_corners_ = {track_id: pts_3d[1:, :].reshape(-1, 3) for track_id, pts_3d in module_corners.items()}    
    
    module_geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for track_id in module_corners.keys():
        
        # module outlines as polygons
        polygons = []
        for corner in module_corners_[track_id][::-1]:  # polygon must be right-handed
            lat, lon, alt = enu2geodetic(*corner, *gps_origin)
            polygons.append([lon, lat, alt])
        # repeat first corner to form a closed polygon
        corner = module_corners_[track_id][-1]
        lat, lon, alt = enu2geodetic(*corner, *gps_origin)
        polygons.append([lon, lat, alt])
        
        module_geojson["features"].append(
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    polygons
                ]
            },
            "properties": {
                "track_id": track_id
            }
        })
        
        # module centers as points
        lat, lon, alt = enu2geodetic(*module_centers_[track_id].reshape(3,), *gps_origin)
        point = [lon, lat, alt]
        
        module_geojson["features"].append(
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": point
            },
            "properties": {
                "track_id": track_id
            }
        })
    json.dump(module_geojson, open(os.path.join(mapping_root, "module_geolocations.geojson"), "w"))

In [12]:
def pixel_bearings(pts, camera_rotation, camera_matrix):
    """Returns the bearing vectors for given image points.
    
    Args:
        pts (`numpy.ndarray`): Image points (u, v) in pixel coordinates. 
            Shape should be either (-1, 2) or (-1, 1, 2).
        
        rvec (`numpy.ndarray`): Camera rotations matrix (see cv2.Rodrigues).
            It is the rotation which transforms points from image coordinates
            into world coordinates. Note: This is inverse of the OpenCV
            convention in which the transformation maps points from world
            to camera coordinates.
        
        camera_matrix (`numpy.ndarray`): OpenCv camera matrix in pixel units.
        
    Returns:
        bearings (`numpy.ndarray`): Bearing vector for each point. Array of 
            shape (-1, 3).
    """
    # compute bearing vectors    
    c = np.array([camera_matrix[0, 2], camera_matrix[1, 2]])
    f = np.array([camera_matrix[0, 0], camera_matrix[1, 1]])
    
    # compute bearing for each point
    pts = pts.reshape(-1, 2)
    b = np.ones((pts.shape[0], 3))
    b[:, :2] = (pts - c) / f
    b /= np.linalg.norm(b, axis=1).reshape(-1, 1)
    
    # rotate bearings by camera rotation
    b = np.matmul(camera_rotation, b.T).T
    return b


def angle_between_vectors(u, v):
    """Returns the angle between two vectors of vectors in radians.
    
    Args:
        u (`numpy.ndarray`): Shape (-1, 3). Each row is a separate
            3D-vector.
            
        v (`numpy.ndarray`): Shape (-1, 3). Each row is a separate
            3D-vector.
            
    Returns:
        angles (`numpy.ndarray`): Shape (-1,). The ith entry corresponds
            to the angle (in radians) between the ith row in u and v.
    """
    u = u.reshape(-1, 3)
    v = v.reshape(-1, 3)
    s1 = np.diagonal(np.matmul(u, u.T))
    s2 = np.diagonal(np.matmul(v, v.T))
    c = np.diagonal(np.matmul(u, v.T)) / np.sqrt(s1 * s2)    
    c[np.abs(c) >= 1.0] = 0.0
    c[np.abs(c) < 1.0] = np.arccos(c)
    return c


def triangulate_points(pts1, pts2, R1, t1, R2, t2, camera_matrix1, camera_matrix2,
        reproj_thres=5.0, min_ray_angle_degrees=1.0):
    """Triangulate 3D map points from corresponding points in two
    keyframes. R1, t1, R2, t2 are the rotation and translation of
    the two key frames w.r.t. to the map origin. Also check for
    the reprojection error in both frames.
    """
    # if any ray angle is smaller than threshold, mark triangulation as failed
    if min_ray_angle_degrees is not None:
        b1 = pixel_bearings(pts1, R1, camera_matrix1)
        b2 = pixel_bearings(pts2, R2, camera_matrix2)
        angles = angle_between_vectors(b1, b2) * 180.0/math.pi
        if np.sum(angles < min_ray_angle_degrees):
            #print("Ray angle below threshold")
            return False, np.empty(shape=(0, 3), dtype=np.float64)
    
    # create projection matrices needed for triangulation of 3D points
    proj_matrix1 = np.hstack([R1.T, -R1.T.dot(t1)])
    proj_matrix2 = np.hstack([R2.T, -R2.T.dot(t2)])
    proj_matrix1 = camera_matrix1.dot(proj_matrix1)
    proj_matrix2 = camera_matrix2.dot(proj_matrix2)

    # triangulate new map points based on matches with previous key frame
    pts_3d = cv2.triangulatePoints(proj_matrix1, proj_matrix2, pts1.reshape(-1, 2).T, pts2.reshape(-1, 2).T).T
    pts_3d = cv2.convertPointsFromHomogeneous(pts_3d).reshape(-1, 3)
    
    # if any reprojection error exceeds threshold, mark triangulation as failed
    if reproj_thres is not None:
        for R, t, pts, camera_matrix in [(R1, t1, pts1, camera_matrix1), 
                                         (R2, t2, pts2, camera_matrix2)]:     
            rvec, _ = cv2.Rodrigues(R.T)
            tvec = -R.T.dot(t)
            reproj_pts, _ = cv2.projectPoints(pts_3d, rvec, tvec, camera_matrix, None)
            reproj_err = np.linalg.norm(reproj_pts - pts, axis=2)

            if np.sum(reproj_err > reproj_thres):
                #print("Reprojection error exceeds threshold")
                return False, np.empty(shape=(0, 3), dtype=np.float64)
    
    return True, pts_3d

In [13]:
def load_tracks(tracks_file):
    """Load Tracks CSV file."""
    tracks_per_frame = defaultdict(list)
    tracks_per_id = defaultdict(list)
    with open(tracks_file, newline='', encoding="utf-8-sig") as csvfile:  # specifying the encoding skips optional BOM
        # automatically infer CSV file format
        dialect = csv.Sniffer().sniff(csvfile.readline(), delimiters=",;")
        csvfile.seek(0)
        csvreader = csv.reader(csvfile, dialect)
        for row in csvreader:
            frame_name = row[0]
            mask_name = row[1]
            track_id = row[2]
            tracks_per_frame[frame_name].append((mask_name, track_id))
            tracks_per_id[track_id].append((frame_name, mask_name))
    return tracks_per_frame, tracks_per_id

In [14]:
def get_module_points(metas, track_id, frame_name, mask_name, camera_matrix=None, dist_coeffs=None):
    """Loads module corners and center points. Undistorts points if dist_coeffs and camera_matrix are not None."""
    try:
        meta = metas[(track_id, frame_name, mask_name)]
    except KeyError:
        points = None
        print("Meta data for module {} not found".format(track_id))
    else:
        center = np.array(meta["center"]).reshape(1, 2).astype(np.float64)
        quadrilateral = np.array(meta["quadrilateral"]).reshape(-1, 2).astype(np.float64)
        points = np.vstack((center, quadrilateral))
        
        if dist_coeffs is not None:
            points = cv2.undistortPoints(points, camera_matrix, dist_coeffs, None, camera_matrix)
            
    return points

In [15]:
def triangulate_observations(pose_graph, observations, max_combinations=None):
    """Triangulates PV module corners (and centers) by triangulating
    observations from all possible 2-pairs of frames and computing the
    median points in 3D space.
    
    Args:
        pose_graph (`networkx.Graph`): Pose graph containing camera poses
            in the world frame.
        
        observations (`dict` of `dict`): The outer dictionary is indexed by
            the track_id of the module. The inner dictionary is indexed
            by the frame_name (`str`) and contains a `numpy.ndarray` of
            shape (5, 1, 2) of undistorted pixel coordinates of the PV module 
            corners in the respective frame. The first row is the center point, 
            the other rows correspond to the top-left (tl), tr, br and bl points.
    
        max_combinations (`int` or `None`): If None triangulate points from all
            possible 2-pairs of frames. To limit this number specify the maximum
            number of combinations to try here.
    
    Returns:
        module_corners (`dict`): Triangulated 3D points of each module. Index is 
            the module track_id and value is a `numpy.ndarray` of shape (5, 3).
            The meaning of rows in this array corresponds to the meaning of
            rows in the observations.
    """
    module_corners = {}

    for track_id in tqdm.tqdm_notebook(observations.keys()):
        num_observations = len(observations[track_id])
        frame_names = list(observations[track_id].keys())

        all_combinations = list(combinations(range(num_observations), 2))

        # limit number of combinations to try
        random.shuffle(all_combinations)
        all_combinations = all_combinations[:max_combinations]

        pts_3d = []
        for i, j in all_combinations:

            R1, t1 = from_twist(pose_graph.nodes[frame_names[i]]["pose"])
            R2, t2 = from_twist(pose_graph.nodes[frame_names[j]]["pose"])

            pts1 = observations[track_id][frame_names[i]]
            pts2 = observations[track_id][frame_names[j]]
            
            camera_matrix1 = pose_graph.nodes[frame_names[i]]["camera_matrix"]
            camera_matrix2 = pose_graph.nodes[frame_names[j]]["camera_matrix"]

            valid, pts = triangulate_points(pts1, pts2, R1, t1, R2, t2, camera_matrix1, camera_matrix2)
            if valid:
                pts_3d.append(pts)
            #else:
            #    print("Invalid triangulation for track id {} and combination {} {}".format(track_id, i, j))

        if len(pts_3d) > 0:
            pts_3d = np.median(pts_3d, axis=0)
            module_corners[track_id] = pts_3d
    return module_corners

In [16]:
tracks_per_frame, tracks_per_id = load_tracks(tracks_file)
metas = pickle.load(open(patches_meta_file, "rb"))

# keep only tracks of key frames
tracks_per_id_filtered = defaultdict(list)
for track_id, frame_mask_names in tqdm.tqdm_notebook(tracks_per_id.items()):
    for frame_mask_name in frame_mask_names:
        frame_name, mask_name = frame_mask_name
        if frame_name in pose_graph.nodes:
            tracks_per_id_filtered[track_id].append((frame_name, mask_name))
            
# filter out short tracks
min_track_len = 3
assert min_track_len >= 2, "min_track_len must be >= 2"
tracks_per_id_filtered = {k: v for k, v in tracks_per_id_filtered.items() if len(v) >= min_track_len}

# get module observations
observations = {}
for track_id, frame_mask_names in tqdm.tqdm_notebook(tracks_per_id_filtered.items()):
    observations[track_id] = {}
    for frame_name, mask_name in frame_mask_names:
        camera_matrix = pose_graph.nodes[frame_name]["camera_matrix"]
        dist_coeffs = pose_graph.nodes[frame_name]["dist_coeffs"]
        points = get_module_points(metas, track_id, 
            frame_name, mask_name, camera_matrix, dist_coeffs)
        if points is not None:
            observations[track_id][frame_name] = points

# triangulate 
module_corners = triangulate_observations(pose_graph, observations)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [17]:
save_modules(module_corners)

### Merge Duplicate PV Modules

Principle
  - for each keyframe
  - project 3D points of all modules into the key frames
  - determine which modules are fully visible (i.e. all 5 points are visible)
  - find modules for which the projected points lie very close to another in image space and select those as merging candidates

In [18]:
def get_visible_modules(pose_graph, frame_name, module_corners, image_width, image_height):
    """Obtain track IDs and reprojected 2D points of all modules visible in the specified key frame."""
    visible_modules = []
    
    # project all modules into key frame
    R, t = from_twist(pose_graph.nodes[frame_name]["pose"])
    camera_matrix = pose_graph.nodes[frame_name]["camera_matrix"]
    for track_id, pts_3d in module_corners.items():
        rvec, _ = cv2.Rodrigues(R.T)
        tvec = -R.T.dot(t)
        reproj_pts, _ = cv2.projectPoints(pts_3d, rvec, tvec, camera_matrix, None)
        reproj_pts = reproj_pts.reshape(-1, 2)
        
        # determine if module is fully visible
        if (all(reproj_pts[:, 0] >= 0.0) 
            and all(reproj_pts[:, 0] < image_width) 
            and all(reproj_pts[:, 1] >= 0.0) 
            and all(reproj_pts[:, 1] < image_height)):
            
            visible_modules.append((track_id, reproj_pts))
    return visible_modules

In [19]:
def merge_list_of_dicts(dicts):
    merged = {}
    for d in dicts:
        merged.update(d)
    return merged

In [20]:
# merge duplicate modules by projecting each module into each keyframe and finding overlapping modules
image_width = 640
image_height = 512
merge_threshold = 20  # px

dists = []

merge_candidates = nx.Graph()
for frame_name in tqdm.tqdm_notebook(pose_graph.nodes):
    
    visible_modules = get_visible_modules(
        pose_graph, frame_name, module_corners, image_width, image_height)
            
    # find overlapping modules in the image
    all_combinations = list(combinations(range(len(visible_modules)), 2))
    for i, j in all_combinations:    
        mean_dist = np.mean(np.linalg.norm(visible_modules[i][1] - visible_modules[j][1], axis=1))
        dists.append(mean_dist)
        if mean_dist < merge_threshold:
            merge_candidates.add_node(visible_modules[i][0])
            merge_candidates.add_node(visible_modules[j][0])
            merge_candidates.add_edge(visible_modules[i][0], visible_modules[j][0])

merge_candidates = [list(track_ids) for track_ids in nx.connected_components(merge_candidates)]

# fuse observations of merge candidates into a single track and store with first track ID
observations_merge_candidates = {}
for track_ids in merge_candidates:
    obs = [observations[track_id] for track_id in track_ids]
    obs = merge_list_of_dicts(obs)
    observations_merge_candidates[track_ids[0]] = obs
    
# re-triangulate
module_corners_updated = triangulate_observations(pose_graph, observations_merge_candidates)
for track_id, pts_3d in module_corners_updated.items():
    module_corners[track_id] = pts_3d
    
# since we stored the updated module corners under first track ID, we can delete all other track IDs
for track_ids in merge_candidates:
    for track_id in track_ids[1:]:
        del module_corners[track_id]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [21]:
save_modules(module_corners)

### Smoothen module corners with graph optimization

In [22]:
corner_merge_threshold = 20  # px

merged_corners = []

# build graph that encodes connections between module corners
for frame_name in tqdm.tqdm_notebook(pose_graph.nodes):
    
    # get reprojected points of all visible modules
    visible_modules = get_visible_modules(
        pose_graph, frame_name, module_corners, image_width, image_height)
    
    track_ids = []
    pts = []
    for track_id, reproj_pts in visible_modules:
        track_ids.extend([track_id for _ in range(5)])  # 4 corners + 1 center
        pts.append(reproj_pts)
        #plt.scatter(reproj_pts[:, 0], reproj_pts[:, 1])
    #plt.show()
    
    if len(pts) > 0:
        pts = np.vstack(pts)

        # determine which points are close to another
        dist = squareform(pdist(pts))
        dist = np.triu(dist)
        dist[dist == 0] = np.inf
        idxs = np.where(dist < corner_merge_threshold)

        for i, j in zip(idxs[0], idxs[1]):
            merged_corners.append((track_ids[i], track_ids[j], i%5, j%5))
            
            
# built lookup table (track_id, pts_idx) -> unique vertex_id (int) and reverse
track_to_vertex_id = {}
vertex_id_to_track = {}
for i, (track_id1, track_id2, pts_idx1, pts_idx2) in enumerate(merged_corners):
    if (track_id1, pts_idx1) not in track_to_vertex_id.keys():
        track_to_vertex_id[(track_id1, pts_idx1)] = 2*i   
        vertex_id_to_track[2*i] = (track_id1, pts_idx1)
        
    if (track_id2, pts_idx2) not in track_to_vertex_id.keys():
        track_to_vertex_id[(track_id2, pts_idx2)] = 2*i + 1    
        vertex_id_to_track[2*i+1] = (track_id2, pts_idx2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [23]:
# setup optimizer and camera parameters
optimizer = g2o.SparseOptimizer()
solver = g2o.BlockSolverSE3(g2o.LinearSolverCholmodSE3())
solver = g2o.OptimizationAlgorithmLevenberg(solver)
optimizer.set_algorithm(solver)

# add 3D corner points and edges
vertex_ids_added = []
for track_id1, track_id2, pts_idx1, pts_idx2 in tqdm.tqdm_notebook(merged_corners):
    
    vertex_id1 = track_to_vertex_id[(track_id1, pts_idx1)]
    if vertex_id1 not in vertex_ids_added:
        v1 = g2o.VertexPointXYZ()        
        v1.set_id(vertex_id1)
        pts_3d1 = module_corners[track_id1][pts_idx1]
        v1.set_estimate(pts_3d1)
        v1.set_fixed(False)
        optimizer.add_vertex(v1)
        vertex_ids_added.append(vertex_id1)
    
    vertex_id2 = track_to_vertex_id[(track_id2, pts_idx2)]
    if vertex_id2 not in vertex_ids_added:
        v2 = g2o.VertexPointXYZ()
        v2.set_id(vertex_id2)
        pts_3d2 = module_corners[track_id2][pts_idx2]
        v2.set_estimate(pts_3d2)
        v2.set_fixed(False)
        optimizer.add_vertex(v2)
        vertex_ids_added.append(vertex_id2)
    
    # add edge
    edge = g2o.EdgePointXYZ()
    edge.set_vertex(0, optimizer.vertex(vertex_id1))
    edge.set_vertex(1, optimizer.vertex(vertex_id2))
    edge.set_measurement(np.array([0, 0, 0]))
    edge.set_information(np.identity(3)) # TODO: give lower confidence to camera z coordinate, not easy as world z != camer z
    edge.set_robust_kernel(g2o.RobustKernelHuber(np.sqrt(5.99)))
    optimizer.add_edge(edge)
    
print("Number of vertices: {}".format(len(optimizer.vertices())))
print("Number of edges: {}".format(len(optimizer.edges())))

optimizer.initialize_optimization()
optimizer.set_verbose(True)
optimizer.optimize(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



Number of vertices: 24727
Number of edges: 202124


10

In [24]:
# write back result
module_corners_estimated = copy.deepcopy(module_corners)
for vertex_id, vertex in optimizer.vertices().items():
    pts_estimated = vertex.estimate()
    track_id, pts_idx = vertex_id_to_track[vertex_id]
    module_corners_estimated[track_id][pts_idx] = pts_estimated
    #print(vertex_id, pts_estimated, module_corners[track_id][pts_idx], 
    #      np.linalg.norm(pts_estimated - module_corners[track_id][pts_idx]))

In [25]:
# move module center points into the center of the four corners
for track_id in module_corners_estimated.keys():
    module_corners_estimated[track_id][0, :] = np.mean(module_corners_estimated[track_id][1:, :], axis=0)

In [26]:
save_modules(module_corners_estimated)
save_modules_geocoords(module_corners_estimated, overall_origin)

### Camera positions to WGS-84 coordinates

In [27]:
# gps origin is the same for all partial reconstructions
gps_positions = []
for frame_name in pose_graph.nodes:
    pose = pose_graph.nodes[frame_name]["pose"]
    t = pose[3:]
    gps_positions.append(np.array(enu2geodetic(*t, *overall_origin)))
gps_positions = np.array(gps_positions)

In [28]:
import os
import simplekml
gps_positions_new = np.copy(gps_positions)
gps_positions_new[:, 0] = gps_positions[:, 1]
gps_positions_new[:, 1] = gps_positions[:, 0]
gps_positions_new[:, 2] = gps_positions[:, 2]

kml_file = simplekml.Kml()
kml_file.newlinestring(name="trajectory", coords=gps_positions_new)
kml_file.save(os.path.join(mapping_root, "gps.kml"))